1⃣️【第十二周作业】

1. 利用上周NER模型训练任务代码，复现课堂案例中：动态学习率、混合精度、DDP训练实现。
2. 利用课堂案例，实现分布式DDP模型训练。存盘后加载实现推理。



In [28]:
!pip install evaluate seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import evaluate  
from datasets import load_dataset
import numpy as np
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [30]:
# 加载hf中dataset
ds = load_dataset("doushabao4766/msra_ner_k_V3")
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3443
    })
})

In [31]:
for items in ds['train']:
    print(items['tokens'])
    print(items['ner_tags'])
    break

['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [32]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

### 实体映射字典

'O':0   
'B-PER':1   
'I-PER':2   
'B-LOC':3   
'I-LOC':4   
'B-ORG':5   
'I-ORG':6   

In [33]:
# 验证tag标签数量
tags_id = set()
for items in ds['train']:
    tags_id.update(items['ner_tags'])
    
tags_id

{0, 1, 2, 3, 4, 5, 6}

In [34]:
# entity_index
entites = ['O'] + list({'PER', 'LOC', 'ORG'})
tags = ['O']
for entity in entites[1:]:
    tags.append('B-' + entity.upper())
    tags.append('I-' + entity.upper())

entity_index = {entity:i for i, entity in enumerate(entites)}


In [35]:
entity_index

{'O': 0, 'PER': 1, 'ORG': 2, 'LOC': 3}

In [36]:
tags

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [39]:
def data_input_proc(item):
    # 输入文本先拆分为字符，再转换为模型输入的token索引
    batch_texts = [list(text) for text in item['tokens']]
    # 导入拆分为字符的文本列表时，需要设置参数is_split_into_words=True
    input_data = tokenizer(batch_texts, truncation=True, add_special_tokens=False, max_length=512, 
                           is_split_into_words=True, padding='max_length')
    input_data['labels'] = [tag + [0] * (512 - len(tag)) for tag in item['ner_tags']]
    return input_data
ds1 = ds.map(data_input_proc, batched=True)

Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [40]:
ds1.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [41]:
for item in ds1['train']:
    print(item)
    break

{'input_ids': tensor([2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636,  674, 1036, 4997,
        2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768,
        7599, 3198, 8024,  791, 1921, 3300, 3119, 5966,  817,  966, 4638,  741,
         872, 3766,  743, 8024, 3209, 3189, 2218, 1373,  872, 2637,  679, 2496,
        1159, 8013,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 

### 构建模型对象

In [42]:
id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained('bert-base-chinese', 
                                                        num_labels=len(tags),
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)

model.to(DEVICE)
model

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

## 模型手动训练

### transformers动态学习率规划

In [46]:
# dataLoader
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
import torch.optim as optim

train_dl = DataLoader(ds1['train'], shuffle=True, batch_size=16)

# 模型参数分组

param_optimizer = list(model.named_parameters())
bert_params, classifier_params = [], []

for name, params, in param_optimizer:
    if 'bert' in name:
        bert_params.append(params)
    else:
        classifier_params.append(params)

param_groups = [
    {"params":bert_params, "lr":1e-5},
    {"params":classifier_params,"weight_decay":0.1, "lr":1e-3}
]

# optimizer
optimizer = optim.Adam(param_groups)

# 学习率调度器
train_steps = len(train_dl) * 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=100, 
                                            num_training_steps=train_steps)

In [48]:
from tqdm import tqdm

for epoch in range(5):
    model.train()
    tpbar = tqdm(train_dl)
    for items in tpbar:
        items = {k:v.to(DEVICE) for k,v in items.items()}
        optimizer.zero_grad()
        outputs = model(**items)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
    
        tpbar.set_description(f'Epoch:{epoch+1} ' + 
                          f'bert_lr:{scheduler.get_lr()[0]} ' + 
                          f'classifier_lr:{scheduler.get_lr()[1]} '+
                          f'Loss:{loss.item():.4f}')

Epoch:1 bert_lr:2.0000000000000002e-07 classifier_lr:2e-05 Loss:1.3282:   0%|          | 2/2813 [00:04<1:54:28,  2.44s/it]


KeyboardInterrupt: 

## 支持混合精度训练

In [51]:
# dataLoader
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
import torch.optim as optim

train_dl = DataLoader(ds1['train'], shuffle=True, batch_size=16)


# 模型创建
id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained('bert-base-chinese', 
                                                        num_labels=len(tags),
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)

model.to(DEVICE)

# 模型参数分组
param_optimizer = list(model.named_parameters())
bert_params, classifier_params = [],[]

for name,params in param_optimizer:
    if 'bert' in name:
        bert_params.append(params)
    else:
        classifier_params.append(params)

param_groups = [
    {'params':bert_params, 'lr':1e-5},
    {'params':classifier_params, 'weight_decay':0.1, 'lr':1e-3}
]

# optimizer
optimizer = optim.AdamW(param_groups) # 优化器

# 学习率调度器
train_steps = len(train_dl) * 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=100, 
                                            num_training_steps=train_steps)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
from tqdm import tqdm
import torch

DEVICE='cuda'

# 梯度计算缩放器
scaler = torch.GradScaler()

for epoch in range(5):
    model.train()
    tpbar = tqdm(train_dl)
    for items in tpbar:
        items = {k:v.to(DEVICE) for k,v in items.items()}
        optimizer.zero_grad()

        with torch.autocast(device_type='cuda'):
            outputs = model(**items)
        loss = outputs.loss

        # 缩放loss后，调用backward
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
    
        tpbar.set_description(f'Epoch:{epoch+1} ' + 
                          f'bert_lr:{scheduler.get_lr()[0]} ' + 
                          f'classifier_lr:{scheduler.get_lr()[1]} '+
                          f'Loss:{loss.item():.4f}')

Epoch:1 bert_lr:1.7000000000000002e-06 classifier_lr:0.00017 Loss:2.0197:   1%|          | 17/2813 [00:07<21:41,  2.15it/s]               


KeyboardInterrupt: 

In [57]:
%%writefile ddp_simple.py

import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler

# 设置分布式环境
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

# 清理分布式环境
def cleanup():
    dist.destroy_process_group()

# 定义训练循环
def train(rank, world_size):
    setup(rank, world_size)
    
    # 使用更小的模型，如 ResNet18 或 MobileNetV3
    model = models.resnet18(pretrained=False, num_classes=10).to(rank)
    ddp_model = DDP(model, device_ids=[rank])
    
    # 损失函数及优化器
    criterion = nn.CrossEntropyLoss().to(rank)
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.01)
    
    # 定义数据集Dataset的转换和图像增强
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    # 分布式训练采样器
    sampler = DistributedSampler(dataset, num_replicas=world_size, rank=rank)
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=8)  # 原为 16，减小 batch size
    
    # 混合精度 scaler
    scaler = torch.cuda.amp.GradScaler()
    
    for epoch in range(10):
        ddp_model.train()
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(rank), labels.to(rank)
            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast():  # 启用混合精度
                outputs = ddp_model(inputs)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            print(f"Rank {rank}, Epoch {epoch}, Loss: {loss.item():.4f}")

    cleanup()
    
def main():
    world_size = torch.cuda.device_count()
    if world_size < 1:
        raise RuntimeError("至少需要一个可用的 GPU")
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True)

if __name__ == "__main__":
    main()

Overwriting ddp_simple.py


In [58]:
!python ddp_simple.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  w